### Bibliotecas e Funções

In [ ]:
import pandas as pd
from scipy import stats
from math import sqrt, ceil


In [ ]:
def calc_metrics(df, column):
    return df[column].mean(), df[column].median(), df[column].std()

In [ ]:
def calc_confidence_interval(t_student_value, mean, std, n ):
    lower = mean - (t_student_value) * (std)/sqrt(n)
    upper = mean + (t_student_value) * (std)/sqrt(n)
    
    return (lower, upper)


In [ ]:
def calc_non_normal_confidence_interval(df, column, mean, confidence, n):
    se = stats.sem(df[column])
    h = se * stats.t._ppf((1+confidence)/2., n-1)
    
    return (mean - h, mean + h)


In [153]:
def calc_v_t_student(std1, std2, n1, n2=None):
    n2 = n1 if n2 is None else n2

    f1 = std1**2/n1
    f2 = std2**2/n2

    return ceil((f1+f2)**2/((f1**2/(n1-1))+(f2**2/(n2-1))))


In [154]:
def calc_confidence_interval(t_student_value, mean1, mean2, std1, std2, n1, n2=None):
    n2 = n1 if n2 is None else n2

    f1 = std1**2/n1
    f2 = std2**2/n2

    lower = mean1 - mean2 - t_student_value * sqrt(f1 + f2)
    upper = mean1 - mean2 + t_student_value * sqrt(f1 + f2)

    return (lower, upper)


### 1 - Análise da internet sem usar dados ativamente

Método de Medição: Medição realizada via Wifi, uma vez que meus disposivos não possuem conectividade ethernet

Plano contratado: VERO - 680MB

Medidor Utilizado: [Speed Test](https://www.speedtest.net/)


In [ ]:
idle = pd.read_csv('./data/erick/parado.csv')
idle

In [ ]:
download_mean, download_median, download_std = calc_metrics(idle, 'speed_download')

print(f'Média de Download: {download_mean}')
print(f'Mediana de Download: {download_median}')
print(f'Desvio Padrão: {download_std}')


In [ ]:
upload_mean, upload_median, upload_std = calc_metrics(idle, 'speed_upload')

print(f'Média de Upload: {upload_mean}')
print(f'Mediana de Upload: {upload_median}')
print(f'Desvio Padrão: {upload_std}')

##### Considerando Distribuição Normal

Para o cálculo da média populacional vamos empregar o caso 4, presente nos slides:

Intervalo de confiança para a média populacional µ de uma distribuição normal com a variância populacional σ² desconhecida e amostra pequena:

In [ ]:
confidence = .95
alpha = 1 - confidence

n = len(idle["speed_download"])
print(f't{round(alpha/2,3)}; {n-1}')

Buscamos o valor acima na tabela T-Student e dela retiramos 2.145

In [ ]:
download_confidence = calc_confidence_interval(2.145, download_mean, download_std, n)

print(download_confidence)

In [ ]:
upload_confidence = calc_confidence_interval(2.145, upload_mean, upload_std, n)

print(upload_confidence)

##### Sem Considerar Distribuição Normal

In [ ]:
n_download_confidence = calc_non_normal_confidence_interval(idle, 'speed_download', download_mean, confidence, n)

print(n_download_confidence)

In [ ]:
n_upload_confidence = calc_non_normal_confidence_interval(idle, 'speed_upload', upload_mean, confidence, n)

print(n_upload_confidence)

### 2 - Comparando Medidores

Método de Medição: Medição realizada via Wifi, uma vez que meus disposivos não possuem conectividade ethernet

Plano contratado: VERO - 680MB

Medidores Utilizados: [Fast](https://fast.com/pt/#), [N Perf](https://www.nperf.com/pt/)


#### Medidor Fast

In [162]:
fast_download_mean, fast_download_median, fast_download_std = calc_metrics(idle, 'fast_download')

print(f'Média de Download: {fast_download_mean}')
print(f'Mediana de Download: {fast_download_median}')
print(f'Desvio Padrão: {fast_download_std}\n')

fast_upload_mean, fast_upload_median, fast_upload_std = calc_metrics(idle, 'fast_upload')

print(f'Média de Upload: {fast_upload_mean}')
print(f'Mediana de Upload: {fast_upload_median}')
print(f'Desvio Padrão: {fast_upload_std}')



Média de Download: 270.6666666666667
Mediana de Download: 270.0
Desvio Padrão: 94.90345971916234

Média de Upload: 165.73333333333332
Mediana de Upload: 180.0
Desvio Padrão: 71.97962674720304


#### Medidor nPerf


In [161]:
nperf_download_mean, nperf_download_median, nperf_download_std = calc_metrics(idle, 'nperf_download')

print(f'Média de Download: {nperf_download_mean}')
print(f'Mediana de Download: {nperf_download_median}')
print(f'Desvio Padrão: {nperf_download_std}\n')

nperf_upload_mean, nperf_upload_median, nperf_upload_std = calc_metrics(idle, 'nperf_upload')

print(f'Média de Upload: {nperf_upload_mean}')
print(f'Mediana de Upload: {nperf_upload_median}')
print(f'Desvio Padrão: {nperf_upload_std}')

Média de Download: 253.66
Mediana de Download: 263.6
Desvio Padrão: 38.01473022771192

Média de Upload: 284.12
Mediana de Upload: 267.3
Desvio Padrão: 36.16672108839443


In [ ]:
n = len(idle['fast_download'])
confidence = .95

$$H0: Download\;Fast = Download\;NPerf$$
$$H1: Download\;Fast > Download\;NPerf$$

In [155]:
v = calc_v_t_student(fast_download_std, nperf_download_std, n)
print(f't{round(1-confidence,3)}; {v}')

t0.05; 19


Buscamos o valor acima na tabela T-Student e dela retiramos 1.729.

Calculando o intervalo de confiança unilateral obtemos:

In [158]:
download_comp_interval = calc_confidence_interval(1.729, fast_download_mean, nperf_download_mean, nperf_download_std, fast_download_std, n)
print(download_comp_interval)

(-28.633227235094793, 62.64656056842817)


$$H0: Upload\;NPerf = Upload\;Fast$$
$$H1: Upload\;NPerf > Upload\;Fast$$

In [163]:
v = calc_v_t_student(nperf_upload_std, fast_upload_std, n)
print(f't{round(1-confidence,3)}; {v}')

t0.05; 21


Buscamos o valor acima na tabela T-Student e dela retiramos 1.721.

Calculando o intervalo de confiança unilateral obtemos:

In [164]:
download_comp_interval = calc_confidence_interval(1.721, nperf_upload_mean, fast_upload_mean, nperf_upload_std, fast_upload_std, n)
print(download_comp_interval)

(82.59125016516622, 154.18208316816714)
